In [78]:
#importing library
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import optuna
import xgboost as xgb
from sklearn.metrics import r2_score
import re

In [52]:
#loading the dataset
df=pd.read_csv('/kaggle/input/SolarEnergy/SolarPrediction.csv')
#showing the dataset
df

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [53]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


In [54]:
#total missing value
print("Total missing values",df.isna().sum().sum())

Total missing values 0


In [55]:
#splitting the day month and year column from date time column
df['Month']=df['Data'].apply(lambda x:re.search(r'^\d+',x).group(0)).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [56]:
#splitting the day column from the table
df['Day']=df['Data'].apply(lambda x:re.search(r'(?<=\/)\d+(?=\/)',x).group(0)).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [58]:
#extracting the year part of date time column
df['Year']=df['Data'].apply(lambda x:re.search(r'(?<=\/)\d+(?=\s)',x).group(0)).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [59]:
#dropping the unnecessary column
df=df.drop('Data',axis=1)

In [ ]:
df

In [60]:
df['Hour']=df['Time'].apply(lambda x:re.search(r'^\d+',x).group(0)).astype(np.int)
df['Minute']=df['Time'].apply(lambda x:re.search(r'(?<=:)\d+(?=:)',x).group(0)).astype(np.int)
df['Second']=df['Time'].apply(lambda x:re.search(r'\d+$',x).group(0)).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the releas

In [61]:
df['SunriseHour']=df['TimeSunRise'].apply(lambda x:re.search(r'^\d+',x).group(0)).astype(np.int)
df['SunriseMinute']=df['TimeSunRise'].apply(lambda x:re.search(r'(?<=:)\d+(?=:)',x).group(0)).astype(np.int)
df['SunsetHour']=df['TimeSunSet'].apply(lambda x:re.search(r'^\d+',x).group(0)).astype(np.int)
df['SunsetMinute']=df['TimeSunSet'].apply(lambda x:re.search(r'(?<=:)\d+(?=:)',x).group(0)).astype(np.int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the releas

In [75]:
#df=df.drop(['TimeSunRise','TimeSunSet'],axis=1)

In [64]:
df

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,SunriseHour,SunriseMinute,SunsetHour,SunsetMinute
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,00:20:04,1.22,44,30.43,102,145.42,6.75,12,1,2016,0,20,4,6,41,17,42
32682,1480587301,00:15:01,1.17,44,30.42,102,117.78,6.75,12,1,2016,0,15,1,6,41,17,42
32683,1480587001,00:10:01,1.20,44,30.42,102,145.19,9.00,12,1,2016,0,10,1,6,41,17,42
32684,1480586702,00:05:02,1.23,44,30.42,101,164.19,7.87,12,1,2016,0,5,2,6,41,17,42


In [68]:
df=df.drop('Time',axis=1)

In [69]:
#Splitting the data
y=df['Radiation'].copy()
x=df.drop('Radiation',axis=1).copy()


In [70]:
scaler=StandardScaler()
x=scaler.fit_transform(x)

In [72]:
x

array([[-0.93775298, -0.50043927,  0.67897416, ..., -0.94342502,
         1.36912575, -1.52576493],
       [-0.93785381, -0.50043927,  0.67897416, ..., -0.94342502,
         1.36912575, -1.52576493],
       [-0.93795264, -0.50043927,  0.67897416, ..., -0.94342502,
         1.36912575, -1.52576493],
       ...,
       [ 0.84517269, -1.14549002, -0.05265751, ...,  0.86087583,
        -0.7303931 ,  0.29376621],
       [ 0.84507319, -1.14549002, -0.05265751, ...,  0.86087583,
        -0.7303931 ,  0.29376621],
       [ 0.84497336, -1.14549002,  0.1302504 , ...,  0.86087583,
        -0.7303931 ,  0.29376621]])

In [74]:
#train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)


In [76]:
print(x_train)
print(x_test)
print(y_train)
print(y_test)

[[ 1.10216118 -0.50043927  0.1302504  ...  1.18307241 -0.7303931
   0.41925111]
 [ 0.45192721  0.95092491  0.49606624 ...  0.28092198 -0.7303931
   0.35650866]
 [-0.29632203 -0.50043927 -0.23556543 ... -0.49234981 -0.7303931
   1.04667565]
 ...
 [-1.47316462  1.27345028  1.41060583 ... -1.20118229  1.36912575
  -0.52188568]
 [ 0.07196463 -1.14549002  0.1302504  ... -0.17015323 -0.7303931
   0.60747847]
 [ 0.54776744 -0.01665121 -0.60138127 ...  0.40980061 -0.7303931
   0.29376621]]
[[ 0.25655356  1.91850103  0.49606624 ...  0.02316472 -0.7303931
   0.48199357]
 [ 1.3303742  -0.66170196 -2.06464461 ...  1.50526899 -0.7303931
   0.60747847]
 [ 1.49061163  0.95092491  0.86188208 ...  1.69858693 -0.7303931
   0.79570583]
 ...
 [ 0.36736988  0.14461147  1.22769791 ...  0.21648267 -0.7303931
   0.35650866]
 [ 0.69053028  0.14461147  0.1302504  ...  0.60311856 -0.7303931
   0.29376621]
 [ 0.05379486  2.2410264   0.31315832 ... -0.17015323 -0.7303931
   0.60747847]]
30858      1.24
19824    31

In [79]:
model=LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [81]:
model.score(x_test,y_test)

0.6283239126911679